In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
factories = 5

In [3]:
ship_prices = np.array([84, 36, 42, 81, 63, 60, 66, 78, 15, 42, 
                                    63, 48, 33, 24, 33, 21, 33, 33, 60, 60,
                                    63, 18, 33, 66, 45, 45, 51, 60, 9, 24,
                                    39, 33, 57, 63, 42, 51, 45, 51, 45, 15,
                                    30, 21, 42, 42, 24, 33, 24, 33, 39, 12])

In [4]:
variables = len(ship_prices)

In [23]:
reserves = np.array([290, 165, 235, 255, 105])

In [24]:
demand = np.array([90, 65, 45, 75, 95, 100, 75, 150, 150, 205])

In [18]:
def matrix_of_reserves_limitations(factories, variables):
    
    res = np.zeros(shape=(factories, variables), dtype=int)
    count = variables / factories     #count = 10
    
    for index in range(0, factories):
        begin = count * index   #откуда начинать
        temp = np.zeros(50)
        temp[int(begin):int(begin+count)] = 1
        res[index] = temp
        
        
    return res

In [19]:
limitations_to_reserves=matrix_of_reserves_limitations(factories, variables)
limitations_to_reserves

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1]])

In [20]:
def matrix_of_demand_limitations(factories, variables):
    count = int(variables / factories)
    res = np.zeros(shape=(count, variables), dtype=int) #пустая матрица из 0         
    begin = 0
    while begin < 9:
        for index in range(count):
            temp = np.zeros(variables, dtype=int)
            indexTemp = 0
            for index2 in range(0,variables,count):
                temp[int(indexTemp)] = 1
                indexTemp += count
            begin+=1
            res[index] = temp
    return res

In [21]:
limitations_to_demand=matrix_of_demand_limitations(factories, variables)

In [22]:
linprog(c=ship_prices, 
              A_ub=limitations_to_reserves, b_ub=reserves,
              A_eq=limitations_to_demand, b_eq=demand)

     con: array([ 90.,  65.,  45.,  75.,  95., 100.,  75., 150., 150., 205.])
     fun: 0.0
 message: 'There is a linear combination of rows of A_eq that results in zero, suggesting a redundant constraint. However the same linear combination of b_eq is nonzero, suggesting that the constraints conflict and the problem is infeasible.'
     nit: 0
   slack: array([290., 165., 235., 255., 105.])
  status: 2
 success: False
       x: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])